In [7]:
import tensorflow as tf


from IPython.display import clear_output, Image, display, HTML
import numpy as np 
import os

from tensorflow.keras.applications import Xception

In [ ]:
tf.keras.backend.set_learning_phase(0)
model = Xception()

sess = k.get_session()
main_saver = tf.train.Saver()
main_saver.save(sess , "xception_save/xception")

In [3]:
merged_graph = tf.Graph()

with merged_graph.as_default():
    
    input = tf.placeholder(dtype=tf.float32, shape=(None,299,299,3))
    
    saver1 = tf.train.import_meta_graph('xception_save/xception.meta', import_scope='g1',
                                        input_map={"input_1:0": input})
    saver2 = tf.train.import_meta_graph('xception_save/xception.meta', import_scope='g2',
                                        input_map={"input_1:0": input})

    output1 = merged_graph.get_tensor_by_name("g1/predictions/Softmax:0")
    
    output2 = merged_graph.get_tensor_by_name("g2/predictions/Softmax:0")
    
    final_output = (output1 + output2) / 2

In [18]:
with tf.Session(graph=merged_graph) as sess:
    saver1.restore(sess, 'xception_save/xception')
    saver1.restore(sess, 'xception_save/xception')
    # this line should NOT run because it will initialize all variables, your restore op will have no effect
    # sess.run(tf.global_variables_initializer())
    
    saver_merged = tf.train.Saver()
    dummy_input = np.random.randn(1,299,299,3)
    final_output_numpy = sess.run(final_output, feed_dict={input: dummy_input})
    
    saver_merged.save(sess , "xception_merged/xcep")


INFO:tensorflow:Restoring parameters from xception_save/xception
INFO:tensorflow:Restoring parameters from xception_save/xception
Instructions for updating:
Colocations handled automatically by placer.


In [15]:
final_output_numpy.shape

(1, 1000)

In [17]:
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
show_graph(merged_graph.as_graph_def())